In [1]:
import pandas as pd
import os
os.chdir('..')
os.getcwd()

'/home/linh/projects/IConNet'

In [2]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
import gc
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
!nvidia-smi

Sun Feb 25 17:48:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080        On  | 00000000:01:00.0  On |                  N/A |
| 32%   39C    P8              19W / 215W |    989MiB /  8192MiB |      9%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
audio_dir = '../data/ESD/full_release/'

In [5]:
import glob

In [9]:
a = glob.glob(f'{audio_dir}001*/{labels[0]}/*.wav')
len(a)

3500

In [12]:
import soundfile as sf

def load_audio(filename, audio_dir=""):
    waveform, sample_rate = sf.read(f'{audio_dir}{filename}') 
    return waveform

In [13]:
load_audio(a[0])

array([-0.00048828, -0.00045776, -0.00048828, ...,  0.00073242,
        0.00073242,  0.0007019 ])

In [14]:
waveform, sample_rate = sf.read(a[0])
print(waveform.shape)
print(sample_rate)

(46400,)
16000


In [15]:
# speaker/emo/speaker_sentence.wav
# speaker (0001-0010: NonEng; 0011-0020: Eng)
# emo: Angry, Neutral, Sad, Happy, Surprise
data_path = glob.glob(f'{audio_dir}*/*/*.wav')
len(data_path)

35000

In [17]:
filepath, filename = os.path.split(data_path[0])
filepath.split('/')

['..', 'data', 'ESD', 'full_release', '0017', 'Angry']

In [31]:
'/'.join(data_path[0].split('/')[-3:])

'0017/Angry/0017_000428.wav'

In [18]:
target_labels = ['anger', 'neutral', 
                 'sadness', 'hapiness', 
                 'surprise']
orig_labels = ['Angry', 'Neutral',
               'Sad', 'Happy',
              'Surprise']

In [19]:
def label_to_target(l):
    return target_labels[orig_labels.index(l)]

In [21]:
label_to_target('Happy')

'hapiness'

In [32]:
df = pd.DataFrame(
    columns=['filepath', 'filename', 'emotion', 'speaker', 'gender', 'language'])

for d in data_path:
    filepath, filename = os.path.split(d)
    fpath = '/'.join(d.split('/')[-3:])
    fp = filepath.split('/')
    speaker = fp[-2]
    emo = fp[-1]
    if int(speaker)%2 == 0:
        gender = 'male'
    else:
        gender = 'female'
    if int(speaker) <= 10:
        language = 'chinese'
    else:
        language = 'english'
    df = df._append({
        'filepath': fpath,
        'filename': filename,
        'emotion': label_to_target(emo),
        'speaker': speaker,
        'gender': gender,
        'language': language
    }, ignore_index=True)    

df

,filepath,filename,emotion,speaker,gender,language
0,0017/Angry/0017_000428.wav,0017_000428.wav,anger,0017,female,english
1,0017/Angry/0017_000422.wav,0017_000422.wav,anger,0017,female,english
2,0017/Angry/0017_000598.wav,0017_000598.wav,anger,0017,female,english
3,0017/Angry/0017_000484.wav,0017_000484.wav,anger,0017,female,english
4,0017/Angry/0017_000429.wav,0017_000429.wav,anger,0017,female,english
...,...,...,...,...,...,...
34995,0009/Surprise/0009_001561.wav,0009_001561.wav,surprise,0009,female,chinese
34996,0009/Surprise/0009_001473.wav,0009_001473.wav,surprise,0009,female,chinese
34997,0009/Surprise/0009_001615.wav,0009_001615.wav,surprise,0009,female,chinese
34998,0009/Surprise/0009_001594.wav,0009_001594.wav,surprise,0009,female,chinese


In [33]:
df.to_csv(
    '../data/data_preprocessed/esd/metadata/data.csv',
    header=True, index=False)

In [34]:
labels = ['ang', 'neu', 'sad', 'hap']
df4 = df[df['emotion'].apply(lambda x: x[:3] in labels)]
df4['emotion'].value_counts()

emotion
anger       7000
sadness     7000
hapiness    7000
neutral     7000
Name: count, dtype: int64

In [35]:
from functools import partial

load_wav = partial(load_audio, audio_dir=audio_dir)
load_wav(df4['filepath'].values[0])

array([-0.00048828, -0.00045776, -0.00048828, ...,  0.00073242,
        0.00073242,  0.0007019 ])

In [36]:
df4['audio'] = df4['filepath'].apply(load_wav)

/tmp/ipykernel_4009961/1211551078.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['audio'] = df4['filepath'].apply(load_wav)


In [37]:
df4['audio'][0].shape

(46400,)

In [38]:
df4['audio'].apply(lambda x: len(x.shape)).value_counts()

audio
1    28000
Name: count, dtype: int64

In [39]:
data_x = df4['audio'].to_list()
data_y = df4['emotion'].to_list()

In [40]:
feature_dir = '../data/data_preprocessed/esd/features_4balanced/'

In [42]:
import numpy as np

In [43]:
np.save(f'{feature_dir}esd.audio16k.npy', 
        np.array(data_x, dtype=object), 
        allow_pickle=True)

In [44]:
np.save(f'{feature_dir}esd.label_emotion.npy', 
        np.array(data_y, dtype=str), 
        allow_pickle=True)